In [1]:
#divergencias
import util
from backtesting import Backtest
import estrategias as est

lista = ['ETHUSDT','BCHUSDT','XRPUSDT','ADAUSDT','FILUSDT','MATICUSDT','OPUSDT','FETUSDT','BTCUSDT'] # lista acotada

#lista=['XRPUSDT']

timeframe = '1h'

###
razon_max = 0.0
razon = 0.0
suma = 0.0
trades = 0
for largo in [1]:
    try:
        suma = 0.0
        trades = 0
        razon = 0.0
        for symbol in lista:
            try:    
                data = est.estrategia_lucho (symbol,timeframe)
                bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
                output = bt.run()
                if len(lista)==1:
                    bt.plot(filename="graficos/"+symbol)
                print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
                suma=suma+output['Return [%]']
                trades=trades+output['# Trades']
            except Exception as e:
                print(f"Error en {symbol}: {e}")
        razon = suma/trades
        if razon > razon_max:
            razon_max = razon
        print(f"Suma total: {suma} - Trades total: {trades}")
        print(f"razon_max: {razon_max}")
    except Exception as e:
        print(f"Error en {symbol}: {e}")
        print(f"Suma total: {suma} - Trades total: {trades}")
        print(f"razon_max: {razon_max}")
print(f"razon_max: {razon_max}")

Symbol: ETHUSDT - Return [%]: 23.7 - # Trades: 34 - Win Rate [%]: 58.82352941176471
Symbol: BCHUSDT - Return [%]: 16.41 - # Trades: 31 - Win Rate [%]: 51.61290322580645
Symbol: XRPUSDT - Return [%]: 17.02 - # Trades: 32 - Win Rate [%]: 53.125
Symbol: ADAUSDT - Return [%]: 19.1 - # Trades: 31 - Win Rate [%]: 58.06451612903226
Symbol: FILUSDT - Return [%]: 15.94 - # Trades: 32 - Win Rate [%]: 53.125
Symbol: MATICUSDT - Return [%]: 16.12 - # Trades: 34 - Win Rate [%]: 50.0
Symbol: OPUSDT - Return [%]: 19.73 - # Trades: 31 - Win Rate [%]: 58.06451612903226
Symbol: FETUSDT - Return [%]: 20.92 - # Trades: 28 - Win Rate [%]: 60.71428571428571
Symbol: BTCUSDT - Return [%]: 17.89 - # Trades: 33 - Win Rate [%]: 54.54545454545454
Suma total: 166.83533120393423 - Trades total: 286
razon_max: 0.5833403188948749
razon_max: 0.5833403188948749
